In [1]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import pickle

ModuleNotFoundError: No module named 'keras'

In [2]:
fileobj = open('train_data.pkl','rb')
trainX, trainy = pickle.load(fileobj)
fileobj.close()
print(trainX.shape, trainy.shape)

(1750, 256, 256, 3) (1750,)


In [3]:
fileobj = open('test_data.pkl','rb')
testX, testy = pickle.load(fileobj)
fileobj.close()
print(testX.shape, testy.shape)

(750, 256, 256, 3) (750,)


In [4]:
pd.read_csv('categries_data.csv')

,Unnamed: 0,0
0,0,Big Red
1,1,Doublemint
2,2,Juicy Fruit
3,3,Spearmint
4,4,Winterfresh


In [5]:
width, height, channels = 256,256,3

In [6]:
# one hot encode target values
trainy = to_categorical(trainy)
testy = to_categorical(testy)

In [7]:
print(trainy.shape)
print(testy.shape)

(1750, 5)
(750, 5)


In [8]:
# confirm scale of pixels
print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))

Train min=0.000, max=255.000
Test min=0.000, max=255.000


In [9]:
# create generator (1.0/255.0 = 0.003921568627451)
datagen = ImageDataGenerator(rescale=1.0/255.0)

In [10]:
# prepare an iterators to scale images
train_iterator = datagen.flow(trainX, trainy, batch_size=64)
test_iterator = datagen.flow(testX, testy, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Batches train=28, test=12


In [11]:
# confirm the scaling works
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(64, 256, 256, 3), min=0.000, max=1.000


In [12]:
# define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# fit model with generator
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=6)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
19/28 [===================>..........] - ETA: 57s - loss: 3.3338 - acc: 0.2220 

KeyboardInterrupt: 

In [ ]:
# evaluate model
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))